In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
import pandas as pd
from datasets import Dataset

csv_path = '/content/drive/MyDrive/clean_data.csv'
df = pd.read_csv(csv_path)

# HuggingFace Dataset으로 변환 후 90/10 split
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.1)

In [ ]:
# import os
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # 환경 변수에서 토큰 읽기
# hf_token = os.getenv("HF_TOKEN")

In [66]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

HF_TOKEN = hf_token

from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "wisenut-nlp-team/KoT5",
    revision="paraphrase",
    use_auth_token= HF_TOKEN
)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "wisenut-nlp-team/KoT5",
    revision="paraphrase",
    use_auth_token= HF_TOKEN
)

In [62]:
# 3. 전처리 함수 (source → target)
max_len = 128

def preprocess(batch):
    # source 컬럼의 텍스트를 입력으로, target 컬럼을 레이블로 사용
    inputs = tokenizer(
        batch["source"],
        truncation=True,
        padding="max_length",
        max_length=max_len
    )
    targets = tokenizer(
        batch["target"],
        truncation=True,
        padding="max_length",
        max_length=max_len
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

# 4. 매핑 실행
tokenized = dataset.map(
    preprocess,
    batched=True,
    remove_columns=["source", "target"]
)

Map:   0%|          | 0/188 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

In [63]:
from transformers import TrainingArguments, Trainer

# 1) 최소한의 TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",            # 체크포인트가 없어도 결과 폴더만 지정
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=90,
    learning_rate=3e-5,
    logging_steps=100,
    fp16=True,
    report_to="none"
)

# 2) Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],    # eval_dataset만 지정
    tokenizer=tokenizer
)

# 3) 학습
trainer.train()

<ipython-input-63-8c0b32b23e62>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,0.156200
200,0.121400
300,0.093400
400,0.077200
500,0.071100
600,0.067500
700,0.069100
800,0.067900
900,0.068600


TrainOutput(global_step=990, training_loss=0.08625466077014653, metrics={'train_runtime': 730.65, 'train_samples_per_second': 23.157, 'train_steps_per_second': 1.355, 'total_flos': 2361538699591680.0, 'train_loss': 0.08625466077014653, 'epoch': 82.51063829787235})

In [64]:
# validation loss, 기타 지표 확인
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.12372644990682602, 'eval_runtime': 0.2487, 'eval_samples_per_second': 84.438, 'eval_steps_per_second': 12.063, 'epoch': 82.51063829787235}


In [ ]:
# 모델 가중치 저장
trainer.save_model("/content/drive/MyDrive/model/kot5-base-finetuned-final")

# 토크나이저 설정 저장
tokenizer.save_pretrained("/content/drive/MyDrive/model/kot5-base-finetuned-final")

('/content/drive/MyDrive/model/kot5-base-finetuned-final/tokenizer_config.json',
 '/content/drive/MyDrive/model/kot5-base-finetuned-final/special_tokens_map.json',
 '/content/drive/MyDrive/model/kot5-base-finetuned-final/tokenizer.json')

In [69]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/content/drive/MyDrive/model/kot5-base-finetuned-final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model     = AutoModelForSeq2SeqLM.from_pretrained(model_path)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0
)

# 예시 테스트
examples = [
  "해지"
]

for src in examples:
    out = pipe(src, max_length=128, min_length=32, do_sample=False)[0]["generated_text"]
    print(f">>> {src}\n→ {out}\n")

Device set to use cuda:0


>>> 해지
→ 돈을 내는 것. 은행이 돈을 내는 것. .

